# Task 1: Automatic classification of tweets that report adverse effects

In [ ]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 80 kB 7.8 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 6.6 MB 35.9 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 287 kB 45.9 MB/s 
     |████████████████████████████████| 106 kB 48.2 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 


## 1. Import all the necessary libraries and data files

In [ ]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from transformers import RobertaTokenizerFast, RobertaModel
from transformers import AutoTokenizer, AutoModel
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [ ]:
train_filename = "task2_en_training.tsv"
val_filename = "task2_en_validation.tsv"

In [ ]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [ ]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (20544, 4) and validation data is (5134, 4)


In [ ]:
# Train top 5 rows
train.head().style.set_caption("Task 1: Train dataset")

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make things much worse...and why give bayer more of your money? they already screwed you once w/ essure"
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made chaos more vengeful and sadistic.
4,345567138376994816,138795534,0,"have decided to skip my #humira shot today. my body's having hysterics, need time to simmer down #rheum"


## 2. Prepare the data - Clean & Prepare for Model

In [ ]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [ ]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [ ]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [ ]:
# Train top 5 rows after pre-processing
train[['class', 'clean_tweets']].head()

,class,clean_tweets
0,0,"depression hurts , cymbalta can help"
1,0,"<user> right , but cipro can make things much worse . <repeated> and why give bayer more of your money ? they already screwed you once w / essure"
2,0,<user> are you on paxil . <repeated> i need help
3,0,<user> the lamotrigine and sjs just made chaos more vengeful and sadistic .
4,0,"have decided to skip my <hashtag> humira </hashtag> shot today . my body ' s having hysterics , need time to simmer down <hashtag> rheum </hashtag>"


In [ ]:
# Define BioBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

In [ ]:
# Tokenize train and validation data
train_enc_bio = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc_bio = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [ ]:
train_enc_bio.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_enc_bio.input_ids.shape, train_enc_bio.token_type_ids.shape, train_enc_bio.attention_mask.shape

(torch.Size([20544, 128]), torch.Size([20544, 128]), torch.Size([20544, 128]))

In [ ]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [ ]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [ ]:
train_dataloader_bio = get_dataloader(train_enc_bio, torch.tensor(train['class'].to_list()))
valid_dataloader_bio = get_dataloader(valid_enc_bio, torch.tensor(validation['class'].to_list()))

In [ ]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader_bio:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - BioBERT

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
class BioBERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(BioBERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [ ]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [ ]:
transformer = AutoModel.from_pretrained('dmis-lab/biobert-base-cased-v1.1')
model = BioBERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model):,} trainable parameters.")

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 108,311,810 trainable parameters.


In [ ]:
# for name, param in model.named_parameters():
#     if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameter(model):,} trainable parameters')

The model has 108,311,810 trainable parameters


In [ ]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

#### Train SciBERT Model

In [ ]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [ ]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [ ]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader_bio)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader_bio)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("\nBest Model Saved!!\n")
    
    # elif epoch % 3 == 0:
    torch.save(model.state_dict(), "model_checkpoint_sci" + str(epoch+1) + ".pt")
    print("\nCheckpoint Model Saved!\n")
    
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.42s/it]
2it [00:02,  1.30s/it]
3it [00:03,  1.26s/it]
4it [00:05,  1.25s/it]
5it [00:06,  1.24s/it]
6it [00:07,  1.23s/it]
7it [00:08,  1.23s/it]
8it [00:09,  1.23s/it]
9it [00:11,  1.23s/it]
10it [00:12,  1.23s/it]
11it [00:13,  1.23s/it]
12it [00:14,  1.23s/it]
13it [00:16,  1.23s/it]
14it [00:17,  1.22s/it]
15it [00:18,  1.23s/it]
16it [00:19,  1.23s/it]
17it [00:21,  1.23s/it]
18it [00:22,  1.23s/it]
19it [00:23,  1.23s/it]
20it [00:24,  1.24s/it]
21it [00:25,  1.23s/it]
22it [00:27,  1.24s/it]
23it [00:28,  1.24s/it]
24it [00:29,  1.24s/it]
25it [00:30,  1.24s/it]
26it [00:32,  1.24s/it]
27it [00:33,  1.24s/it]
28it [00:34,  1.24s/it]
29it [00:35,  1.24s/it]
30it [00:37,  1.24s/it]
31it [00:38,  1.24s/it]
32it [00:39,  1.24s/it]
33it [00:40,  1.24s/it]
34it [00:42,  1.24s/it]
35it [00:43,  1.24s/it]
36it [00:44,  1.24s/it]
37it [00:45,  1.24s/it]
38it [00:47,  1.25s/it]
39it [00:48,  1.25s/it]
40it [00:49,  1.


Best Model Saved!!



 20%|██        | 1/5 [15:04<1:00:18, 904.74s/it]


Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.220 | Train F1 Score: 0.386
Valid Total Loss: 0.176 | Valid F1 Score: 0.558
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.34s/it]
2it [00:02,  1.31s/it]
3it [00:03,  1.30s/it]
4it [00:05,  1.30s/it]
5it [00:06,  1.30s/it]
6it [00:07,  1.29s/it]
7it [00:09,  1.29s/it]
8it [00:10,  1.29s/it]
9it [00:11,  1.29s/it]
10it [00:12,  1.29s/it]
11it [00:14,  1.29s/it]
12it [00:15,  1.29s/it]
13it [00:16,  1.29s/it]
14it [00:18,  1.29s/it]
15it [00:19,  1.29s/it]
16it [00:20,  1.29s/it]
17it [00:21,  1.29s/it]
18it [00:23,  1.29s/it]
19it [00:24,  1.29s/it]
20it [00:25,  1.29s/it]
21it [00:27,  1.29s/it]
22it [00:28,  1.29s/it]
23it [00:29,  1.29s/it]
24it [00:31,  1.29s/it]
25it [00:32,  1.29s/it]
26it [00:33,  1.29s/it]
27it [00:34,  1.29s/it]
28it [00:36,  1.29s/it]
29it [00:37,  1.30s/it]
30it [00:38,  1.30s/it]
31it [00:40,  1.30s/it]
32it [00:41,  1.29s/it]
33it [00:42,  1.29s/it]
34it [00:43,  1.29s/it]
35it [00:45,  1.29s/it]
36it [00:46,  1.29s/it]
37it [00:47,  1.29s/it]
38it [00:49,  1.29s/it]
39it [00:50,  1.29s/it]
40it [00:51,  1.29s/it]
41it [00:52,  1.29s/it]
42it 

...


Checkpoint Model Saved!

Epoch: 03
Train Total Loss: 0.078 | Train F1 Score: 0.865
Valid Total Loss: 0.314 | Valid F1 Score: 0.572
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.27s/it]
2it [00:02,  1.28s/it]
3it [00:03,  1.28s/it]
4it [00:05,  1.28s/it]
5it [00:06,  1.29s/it]
6it [00:07,  1.29s/it]
7it [00:08,  1.29s/it]
8it [00:10,  1.29s/it]
9it [00:11,  1.29s/it]
10it [00:12,  1.29s/it]
11it [00:14,  1.29s/it]
12it [00:15,  1.29s/it]
13it [00:16,  1.29s/it]
14it [00:18,  1.29s/it]
15it [00:19,  1.29s/it]
16it [00:20,  1.29s/it]
17it [00:21,  1.29s/it]
18it [00:23,  1.29s/it]
19it [00:24,  1.29s/it]
20it [00:25,  1.29s/it]
21it [00:27,  1.29s/it]
22it [00:28,  1.29s/it]
23it [00:29,  1.29s/it]
24it [00:30,  1.29s/it]
25it [00:32,  1.29s/it]
26it [00:33,  1.29s/it]
27it [00:34,  1.29s/it]
28it [00:36,  1.29s/it]
29it [00:37,  1.29s/it]
30it [00:38,  1.29s/it]
31it [00:39,  1.29s/it]
32it [00:41,  1.29s/it]
33it [00:42,  1.29s/it]
34it [00:43,  1.29s/it]
35it [00:45,  1.29s/it]
36it [00:46,  1.29s/it]
37it [00:47,  1.29s/it]
38it [00:48,  1.29s/it]
39it [00:50,  1.29s/it]
40it [00:51,  1.29s/it]
41it [00:52,  1.29s/it]
42it 


Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.045 | Train F1 Score: 0.930
Valid Total Loss: 0.399 | Valid F1 Score: 0.551
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.28s/it]
2it [00:02,  1.29s/it]
3it [00:03,  1.29s/it]
4it [00:05,  1.29s/it]
5it [00:06,  1.29s/it]
6it [00:07,  1.29s/it]
7it [00:09,  1.29s/it]
8it [00:10,  1.29s/it]
9it [00:11,  1.30s/it]
10it [00:12,  1.30s/it]
11it [00:14,  1.30s/it]
12it [00:15,  1.30s/it]
13it [00:16,  1.30s/it]
14it [00:18,  1.30s/it]
15it [00:19,  1.30s/it]
16it [00:20,  1.29s/it]
17it [00:22,  1.29s/it]
18it [00:23,  1.29s/it]
19it [00:24,  1.29s/it]
20it [00:25,  1.29s/it]
21it [00:27,  1.29s/it]
22it [00:28,  1.30s/it]
23it [00:29,  1.30s/it]
24it [00:31,  1.29s/it]
25it [00:32,  1.29s/it]
26it [00:33,  1.29s/it]
27it [00:34,  1.29s/it]
28it [00:36,  1.29s/it]
29it [00:37,  1.30s/it]
30it [00:38,  1.30s/it]
31it [00:40,  1.29s/it]
32it [00:41,  1.30s/it]
33it [00:42,  1.30s/it]
34it [00:44,  1.30s/it]
35it [00:45,  1.30s/it]
36it [00:46,  1.29s/it]
37it [00:47,  1.30s/it]
38it [00:49,  1.29s/it]
39it [00:50,  1.30s/it]
40it [00:51,  1.29s/it]
41it [00:53,  1.29s/it]
42it 


Checkpoint Model Saved!

Epoch: 05
Train Total Loss: 0.030 | Train F1 Score: 0.960
Valid Total Loss: 0.404 | Valid F1 Score: 0.587
--------------------


In [ ]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96      4660
           1       0.67      0.48      0.56       474

    accuracy                           0.93      5134
   macro avg       0.81      0.73      0.76      5134
weighted avg       0.92      0.93      0.92      5134



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from glob import glob

In [ ]:
for filepath in glob("*.pt"):
    print(filepath)

model_checkpoint_sci3.pt
model_checkpoint_sci4.pt
model_checkpoint_sci1.pt
model_checkpoint_sci5.pt
model_least_loss.pt
model_checkpoint_sci2.pt


In [ ]:
# !cp -r model_least_loss.pt /content/gdrive/My\ Drive/Colab\ Notebooks/

In [ ]:
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/
    time.sleep(10)